In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.python.keras.applications import vgg16
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras import layers, models, Model, optimizers
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import cv2
import pickle

In [13]:
data_dir = 'E:\EVILDOLL\PYTHON PROJECTS\TENSER FLOW\lol\image_data'
categories = ['manipuri','bharatanatyam','odissi','kathakali','kathak','sattriya','kuchipudi','mohiniyattam']
data = []
hello = []

In [14]:
img_size = 100
def make_data():
    for category in categories:
        path = os.path.join(data_dir, category)
        label = categories.index(category)
        hello.append(category)
        for img_name in os.listdir(path):
            image_path = os.path.join(path, img_name)
            image = cv2.imread(image_path)
            try:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image,(img_size,img_size))
                image = np.array(image, dtype=np.float32)
                data.append([image,label])
            except Exception as e:
                pass  
            
make_data() 

In [15]:
nb_categories = len(hello)
nb_categories

8

In [8]:
img_height, img_width = 224,224
conv_base = vgg16.VGG16(weights='imagenet', include_top=False, pooling='max', input_shape = (img_width, img_height, 3))

In [9]:
for layer in conv_base.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x0000026E992776C8> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000026E9948E388> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000026E99277548> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000026E99ACAE08> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000026E99AAF1C8> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000026E99AB4B48> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000026E99AB7708> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000026E99ABD488> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000026E99464BC8> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000026E99464988> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000026E99478F48> True
<tensorflow.python.keras.layer

In [16]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Dense(nb_categories, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 8)                 4104      
Total params: 14,718,792
Trainable params: 14,718,792
Non-trainable params: 0
_________________________________________________________________


In [17]:
batch_size = 32

train_datagen =  ImageDataGenerator(rescale=1./255)
test_datagen =  ImageDataGenerator(rescale=1./255)

In [18]:
from sklearn.model_selection import train_test_split

In [20]:
def load_data():
    pick = open('data.pickle','rb')
    data = pickle.load(pick)
    pick.close()  
    np.random.shuffle(data)
    features = []
    labels = []
    for img,label in data:
        features.append(img)
        labels.append(label)
        
    features = np.array(features, dtype = np.float32)
    labels = np.array(labels)
    features = features/255.0
    return [features, labels]

In [22]:
features, labels = load_data()

In [23]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=20)

In [31]:
history = model.fit_generator(X_train,y_train ,epochs=10, shuffle=True,validation_data=y_train)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.